##Install Libraries

In [ ]:
!git --version

git version 2.17.1


In [ ]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install audiolazy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 121 kB 5.2 MB/s 


##Import Libraries

In [9]:
import numpy as np
import pandas as pd
import matplotlib
import scipy
import seaborn
import librosa
import audiolazy
import sklearn
import os

import scipy.signal as sig
from audiolazy import lazy_lpc

##Prepare Datasets

In [5]:
# Assign dataset's path to variable
dtsets = '/content/drive/MyDrive/Dataset TA/NewDataPrimer'

emotions = ['bahagia', 'kecewa', 'marah', 'sedih']  # Type of emotion to be classified

In [6]:
os.listdir(dtsets)

['Bahagia', 'Kecewa', 'Sedih', 'Marah']

## Data Augmentation

In [8]:
def noise(signal):
  """
  Adding White Noise.
  """
  #signal, s_rate = lbr.load(wav, duration=2, sr=None)
  noise_amp = 0.01*np.random.uniform()*np.amax(signal)   # more noise reduce the value to 0.5
  signal = signal + noise_amp * np.random.normal(size=signal.shape[0])
  return signal

def stretch(signal, rate=0.8):
  """
  Streching the Sound. Note that this expands the dataset slightly
  """
  #signal, s_rate = lbr.load(wav, duration=2, sr=None)
  signal = librosa.effects.time_stretch(signal, rate)
  return signal

def speedNpitch(signal):
  """
  Speed and Pitch Tuning.
  """
  #signal, s_rate = lbr.load(wav, duration=2, sr=None)
  length_change = np.random.uniform(low=0.8, high = 1)
  speed_fac = 1.2  / length_change
  tmp = np.interp(np.arange(0, len(signal), speed_fac), np.arange(0, len(signal)), signal)
  minlen = min(signal.shape[0], tmp.shape[0])
  signal *= 0
  signal[0:minlen] = tmp[0:minlen]
  return signal

## Linear Predictive Coding (LPC)

In [13]:
def LPC_feature(wav, order= 16):
  # Define LPF Features variable
  lpc_features = []

  # Read speech signal
  signal, s_rate = librosa.load(wav, duration=2, sr=None)

  # Pre-Emphasis
  emphasized_signal = librosa.effects.preemphasis(signal, coef=0.97)

  # Frame Blocking and Windowing
  '''
  frames_win = np.abs(lbr.stft(emphasized_signal)) # using kind of hann window
  idx_frame = len(frames_win)//2
  frames_win = frames_win[idx_frame]
  '''

  # Autocorrelation
  autocorrelation = sig.fftconvolve(emphasized_signal, emphasized_signal[::-1])
  autocorr_coeff = autocorrelation[autocorrelation.size//2:][:(order + 1)]      # Autocorrelation Coefficients

  # LPC features
  lpc_coeff_ld = lazy_lpc.levinson_durbin(autocorr_coeff, 16)
  lpc_features = lpc_coeff_ld.numerator[1:]
  
  return lpc_features

In [14]:
LPC_feature('/content/drive/MyDrive/Dataset TA/NewDataPrimer/Bahagia/Aka Bahagia 1.wav')

[-0.991093778017199,
 1.0949996436974552,
 -0.9159130966813503,
 0.7769950664475589,
 -0.2884422610763104,
 0.43362286110540843,
 -0.08659825610006192,
 0.059157848548422466,
 -0.02303931076473354,
 0.252787595666948,
 -0.07745676398584325,
 0.3397961002147634,
 -0.19199850509235195,
 0.2701888988065858,
 -0.11694606014814908,
 0.17267050992808605]

In [16]:
def load_dataset():
  sounds, emotions = [], []

  for dirname, _, filenames in os.walk(dtsets):
    for filename in filenames:
      #print(os.path.join(dirname, filename))
      sounds.append(os.path.join(dirname, filename))
      #print(filename)
      label = filename.split(' ')[-2]
      #print(label)
      emotions.append(label.lower())
  return {'Audio': sounds, 'Emotion': emotions}

In [17]:
import time

start_time = time.time()
Trial_dict = load_dataset()
print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

--- Data loaded. Loading time: 0.0053293704986572266 seconds ---


In [18]:
audio = Trial_dict['Audio']
label = Trial_dict['Emotion']

data = pd.DataFrame(label, columns = ['Emotion'])
data = pd.concat([data, pd.DataFrame(audio, columns = ['Path'])], axis=1)
data.head(-1)

,Emotion,Path
0,bahagia,/content/drive/MyDrive/Dataset TA/NewDataPrime...
1,bahagia,/content/drive/MyDrive/Dataset TA/NewDataPrime...
2,bahagia,/content/drive/MyDrive/Dataset TA/NewDataPrime...
3,bahagia,/content/drive/MyDrive/Dataset TA/NewDataPrime...
4,bahagia,/content/drive/MyDrive/Dataset TA/NewDataPrime...
...,...,...
74,marah,/content/drive/MyDrive/Dataset TA/NewDataPrime...
75,marah,/content/drive/MyDrive/Dataset TA/NewDataPrime...
76,marah,/content/drive/MyDrive/Dataset TA/NewDataPrime...
77,marah,/content/drive/MyDrive/Dataset TA/NewDataPrime...


##Next